Попробуем предсказать стоимость машин и понять, от каких факторов зависит 
ценообразование на автомобили.
Источник - https://karpov.courses/

In [100]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [74]:
cars = pd.read_csv('cars.csv')
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [75]:
cars.shape #всего в датасете 205 наблюдений
cars.isna().sum() #пропущенных значений нет
cars.dtypes #типы данных соответсвуют

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [76]:
#в датасете приведены полные названия машин - создадим новый столбец с маркой машины

cars["company"] = cars["CarName"].apply(lambda x: x.split(' ')[0])
cars["company"].nunique() # всего встречается 28 уникальных марок
cars.drop(['car_ID', 'CarName'], axis = 1) # также удалим car_ID из датасета

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0,volvo
201,-1,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0,volvo
202,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0,volvo
203,-1,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,idi,3.01,3.40,23.0,106,4800,26,27,22470.0,volvo


In [77]:
cars["company"].value_counts() #названия некоторых марок с ошибками 

toyota         31
nissan         17
mazda          15
mitsubishi     13
honda          13
subaru         12
volvo          11
peugeot        11
volkswagen      9
dodge           9
bmw             8
buick           8
plymouth        7
audi            7
saab            6
porsche         4
isuzu           4
jaguar          3
alfa-romero     3
chevrolet       3
vw              2
maxda           2
renault         2
mercury         1
porcshce        1
toyouta         1
vokswagen       1
Nissan          1
Name: company, dtype: int64

In [78]:
cars["company"] = cars["company"].str.lower() #приведем названияк нижнему регистру
cars["company"] = cars["company"].replace(['maxda', 'porcshce', 'toyouta', 'vokswagen', 'vw'], ['mazda', 'porsche','toyota', 'volkswagen', 'volkswagen']) #заменим на корректные названия
cars["company"].nunique() # всего 22 уникальных марок машин

22

In [79]:
#оставим только нужные предикторы
cars_model = cars[['price','company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower']]
cars_model.head()
cars_model.dtypes

price             float64
company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
dtype: object

In [105]:
cars_model.corr() #посмотрим на корреляцию цены маштины с предикторами

#переменная price наиболее всего коррелирует с переменными horsepower, enginesize,curbweight, carwidth 

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
price,1.000000,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,0.108117,0.324731,...,-0.344270,0.016285,0.385991,-0.004544,0.249606,-0.697762,0.474978,-0.071388,0.199634,-0.004544
wheelbase,0.577816,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.110017,0.147804,...,-0.204037,-0.183195,0.166152,-0.081174,0.261182,-0.309492,0.145842,-0.120709,0.037803,-0.081174
carlength,0.682920,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.149405,0.171120,...,-0.274413,-0.118320,0.244053,-0.057877,0.259894,-0.400210,0.262981,-0.187445,0.100413,-0.057877
carwidth,0.759325,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.246588,0.053412,...,-0.286211,-0.124446,0.348869,-0.013699,0.397690,-0.523135,0.209246,-0.183473,0.153516,-0.013699
curbweight,0.835305,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.088743,0.145028,...,-0.413293,-0.080295,0.400878,-0.039196,0.264554,-0.576463,0.405490,-0.143903,0.187964,-0.039196
enginesize,0.874145,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.017231,0.193885,...,-0.363334,-0.016508,0.562403,-0.184762,0.144878,-0.631431,0.511783,-0.111081,0.335555,-0.184762
boreratio,0.553173,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,-0.109189,0.107399,...,-0.410383,0.326798,0.119509,0.000127,-0.007797,-0.164076,0.128365,-0.108774,0.054482,0.000127
horsepower,0.808139,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.080472,0.177560,...,-0.427616,0.044752,0.431539,0.019250,0.110692,-0.639552,0.533544,-0.099600,0.280220,0.019250
company_audi,0.108117,0.110017,0.149405,0.246588,0.088743,0.017231,-0.109189,0.080472,1.000000,-0.037890,...,0.116687,-0.052831,-0.048926,-0.026525,0.670425,-0.285185,-0.068467,-0.013164,-0.013164,-0.026525
company_bmw,0.324731,0.147804,0.171120,0.053412,0.145028,0.193885,0.107399,0.177560,-0.037890,1.000000,...,0.125060,-0.056621,-0.052436,-0.028428,-0.047985,-0.253894,0.396711,-0.014109,-0.014109,-0.028428


In [106]:
# введем дамми переменные для категориальных переменных
cars_model = pd.get_dummies(data=cars_model, drop_first = True)

In [107]:
cars_model.shape

(205, 49)

In [108]:
# построим линейную модель с одним предиктором - horsepower
smf.ols('price ~ horsepower', data = cars_model).fit().summary()

# переменная значима, модель объясняет 65% изменчивости цены

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           1.48e-48
Time:                        21:46:05   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.000   -5555.163   -1888.360
horsepower   163.2631      8.351     19.549      0.000     146.796     179.730
==============================================================================
Omnibus:                       47.741   Durbin-Watson:                   0.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.702
Skew:                           1.141   Prob(JB):                     1.22e-20
Kurtosis:                       5.352   Cond. No.                         314.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [113]:
# построим модель со всеми предикторами
Y = cars_model['price']
X = cars_model.drop('price',axis = 1)

X = sm.add_constant(X)  
model = sm.OLS(Y, X)  
results = model.fit() 
print(results.summary()) 

#Adj. R-squared в данной модели намного выше, чм в предыдущей = 0.948, при этом большинство переменных связанных с марками машин статистически незначимы

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           4.86e-89
Time:                        21:57:44   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [130]:
# построим модель со всеми предикторами, кроме марок машин
cars_model_wo_price = cars_model.drop('price', axis = 1)
Y = cars_model['price']
X = cars_model_wo_price[cars_model_wo_price.columns.drop(list(cars_model_wo_price.filter(regex='company_')))]
X = sm.add_constant(X)  
model = sm.OLS(Y, X)  
results = model.fit() 
print(results.summary()) 

# Adj. R-squared модели понизился до 0.901, хотя марки машин и объясняют какую-то часть общей дисперсии в данных, эта часть не самая большая - около 5%. 
# При том, что многие переменне связанные с марками машин окались незначимы, можно принять решение оставить третью модель как основную
# Выбранная модель объясняеть примерно 90.1% дисперсии зависимой переменной. Среди 27 предикторов, 10 оказались незначимы.
# Пример интерпретации: при единичном изменении показателя horsepower, цена возрастает на 86.8164 

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Tue, 27 Feb 2024   Prob (F-statistic):           9.86e-81
Time:                        22:09:20   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+